In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "CIFAR10",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': 10000,
            'per_task_examples': np.inf,
            'per_task_memory_examples': 100,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:5' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.002
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=CIFAR10/seed=1_epoch=1_lr=0.001_alpha=0.002_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]
Files already downloaded and verified
Files already downloaded and verified


In [3]:
benchmark.trains[1].inputs.shape

torch.Size([10000, 3, 32, 32])

In [4]:
from algorithms.imbalance import Heuristic2
from algorithms.agem import AGEM
from metrics import MetricCollector2

# backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
backbone = cl.backbones.ResNet18Small(num_classes_per_head = 2, num_classes = 10, config=params)
algorithm = AGEM(backbone, benchmark, params)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [5]:
from trainers.baselines import BaseContinualTrainer

trainer = BaseContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 84.89999999999999, 'loss': 0.0013653969168663026, 'std': 0.20000000000000018}
---------------------------- Task 2 -----------------------
[2] Eval metrics for task 1 >> {'accuracy': 76.64999999999999, 'loss': 0.002127712771296501, 'std': 0.4500000000000004}
[2] Eval metrics for task 2 >> {'accuracy': 75.25, 'loss': 0.0020544551014900206, 'std': 3.150000000000003}
---------------------------- Task 3 -----------------------
[3] Eval metrics for task 1 >> {'accuracy': 74.8, 'loss': 0.0021965804696083067, 'std': 6.1}
[3] Eval metrics for task 2 >> {'accuracy': 71.8, 'loss': 0.0022410117089748383, 'std': 0.30000000000000027}
[3] Eval metrics for task 3 >> {'accuracy': 79.80000000000001, 'loss': 0.0017076421678066255, 'std': 0.7000000000000006}
---------------------------- Task 4 -----------------------
[4] Eval metrics for task 1 >> {'accuracy': 72.35000000000001, 'loss': 0.00219042995572

In [6]:
metric_manager_callback.meters['std'].get_std()

[0.20000000000000018,
 2.3563743335896365,
 4.846189797723114,
 7.417673152680694,
 10.42849941266719]

In [7]:
metric_manager_callback.meters['accuracy'].get_data()

array([[84.9 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [76.65, 75.25,  0.  ,  0.  ,  0.  ],
       [74.8 , 71.8 , 79.8 ,  0.  ,  0.  ],
       [72.35, 67.75, 72.2 , 85.4 ,  0.  ],
       [78.65, 64.85, 60.4 , 71.1 , 88.4 ]])

In [8]:
metric_manager_callback.meters['accuracy'].compute_overall()

[84.89999999999999,
 75.94999999999999,
 75.46666666666667,
 74.425,
 72.67999999999999]